In [1]:
import pandas as pd
import os
os.chdir("C:\\Siddhartha\\Confidential Documents\\Data Science\\MT 599\\")
dataset=pd.read_csv("MT599.csv")

In [2]:
# import dependencies
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import FreqDist

import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
from gensim.models import CoherenceModel

import re

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# Splitting the dataset into the Training set and Test set
df=pd.DataFrame(dataset["Text"])

In [4]:
# document length
df['TextLength'] = df['Text'].str.len()

In [5]:
mean = np.average(df['TextLength'])
sd = np.std(df['TextLength'])

data = df[df['Text'].map(len) >= mean-2*sd]
data = data[data['Text'].map(len) <= mean+2*sd]

In [6]:
def initial_clean(text):
    """
    Function to clean text of websites, email addresess and any punctuation
    We also lower case the text
    """
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub("[^a-zA-Z ]", " ", text)
    text = text.lower() # lower case the text
    text = nltk.word_tokenize(text)
    return text

stop_words = pd.read_csv("stop word list - V1.csv")
stop_words = stop_words['i'].tolist()
def remove_stop_words(text):
    """
    Function that removes all stopwords from text
    """
    return [word for word in text if word not in stop_words]


stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words, so plural and singular are treated the same
    """
    text = [stemmer.stem(word) for word in text]
    text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    
    return text

def apply_all(text):
    """
    This function applies all the functions above into one
    """
    return remove_stop_words(stem_words(remove_stop_words(initial_clean(text))))

In [7]:
# clean text and title and create new column "tokenized"
data['tokenized'] = data['Text'].apply(apply_all)

In [8]:
# first get a list of all words
all_words = [word for item in list(data['tokenized']) for word in item]

# use nltk fdist to get a frequency distribution of all words
fdist = FreqDist(all_words)

In [9]:
k = 10000
# define a function only to keep words in the top k words
top_k_words,_ = zip(*fdist.most_common(k))
top_k_words = set(top_k_words)
def keep_top_k_words(text):
    return [word for word in text if word in top_k_words]

data['tokenized'] = data['tokenized'].apply(keep_top_k_words)

In [10]:
# document length
data['token_len'] = data['tokenized'].apply(lambda x: len(x))

In [11]:
# only keep articles with more than 10 tokens, otherwise too short
data = data[data['tokenized'].map(len) >= 10]
# make sure all tokenized items are lists
data = data[data['tokenized'].map(type) == list]
data.reset_index(drop=True,inplace=True)

In [12]:
#text = data['tokenized']

In [13]:
dictionary = gensim.corpora.Dictionary(data['tokenized'])
dictionary.filter_extremes(keep_n=8000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data['tokenized']]

In [14]:
coherence_lda = []
for i in range(5,25,5):
    for j in range(100,600,200):
        print(i,j)
        lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics= i, id2word=dictionary, random_state=100, chunksize=j, passes=15, workers=4)
        coherence_model_lda = CoherenceModel(model=lda_model, texts=data['tokenized'], dictionary=dictionary, coherence='c_v')
        coherence_lda.append(coherence_model_lda.get_coherence())

5 100
5 300
5 500
10 100
10 300
10 500
15 100
15 300
15 500
20 100
20 300
20 500


In [15]:
Coh_DF = pd.DataFrame([5,5,5,10,10,10,15,15,15,20,20,20])
Chunk_Size = pd.DataFrame([100,300,500,100,300,500,100,300,500,100,300,500])
coherence_lda = pd.DataFrame(coherence_lda)
Coh_DF["Chunk_Size"] = Chunk_Size
Coh_DF["Coherence_Score"] = coherence_lda/100
Coh_DF

,0,Chunk_Size,Coherence_Score
0,5,100,0.004747
1,5,300,0.005010
2,5,500,0.005124
3,10,100,0.005657
4,10,300,0.006646
5,10,500,0.007388
6,15,100,0.006545
7,15,300,0.006224
8,15,500,0.005880
9,20,100,0.006693


In [16]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, random_state=100, chunksize=500, passes=15, workers=4)

In [17]:
# Compute Perplexity
#print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data['tokenized'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print ('\nCoherence Score : %.0f' % (100*coherence_lda) + r'%')


Coherence Score : 74%


In [18]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.047*"mauritiu" + 0.029*"credit" + 0.027*"amount" + 0.026*"valu" + 0.025*"favour" + 0.025*"code" + 0.024*"debit" + 0.024*"asap" + 0.024*"scblmumu" + 0.024*"arrang"
Topic: 1 
Words: 0.061*"cash" + 0.044*"per" + 0.044*"request" + 0.043*"inform" + 0.042*"advis" + 0.040*"open" + 0.026*"record" + 0.026*"deal" + 0.025*"link" + 0.025*"daili"
Topic: 2 
Words: 0.039*"custodi" + 0.031*"cash" + 0.030*"bp" + 0.027*"depositori" + 0.026*"advis" + 0.021*"id" + 0.021*"investor" + 0.021*"th" + 0.017*"winoto" + 0.017*"fmo"
Topic: 3 
Words: 0.065*"address" + 0.052*"swift" + 0.033*"custodian" + 0.030*"settlement" + 0.029*"mark" + 0.028*"twyman" + 0.023*"euroclear" + 0.021*"fedwir" + 0.018*"bkengb" + 0.018*"co"
Topic: 4 
Words: 0.060*"forward" + 0.058*"deal" + 0.043*"amount" + 0.038*"detail" + 0.038*"follow" + 0.033*"advis" + 0.032*"market" + 0.032*"request" + 0.032*"nigeria" + 0.030*"fund"
Topic: 5 
Words: 0.071*"fx" + 0.051*"amount" + 0.051*"note" + 0.028*"request" + 0.025*"remit" + 0.0

In [19]:
import pyLDAvis
import pyLDAvis.gensim

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      19.126962        1       1 -0.204518  0.370943
7      12.245743        1       2 -0.029576 -0.147021
6      10.840320        1       3 -0.253603 -0.127653
3      10.584937        1       4 -0.212970 -0.156939
5       9.994892        1       5  0.178247 -0.023101
8       8.999731        1       6  0.140195 -0.121544
1       8.481300        1       7  0.145155  0.117418
2       7.304006        1       8 -0.025857  0.077665
4       6.476415        1       9  0.213876  0.068799
0       5.945694        1      10  0.049051 -0.058567, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
568   Default  480.000000    audxcci  480.000000  30.0000  30.0000
60    Default  339.000000         fx  339.000000  29.0000  29.0000
97    Default  291.000000    address  291.000000  28.0000  28.0000
63    Default  191.000000    repatri  191.000000  27.0000  27.0000
59    Default  242.000000       fund  242.000000  26.0000  26.0000
19    Default  395.000000        per  395.000000  25.0000  25.0000
72    Default  306.000000       deal  306.000000  24.0000  24.0000
178   Default  222.000000      claim  222.000000  23.0000  23.0000
156   Default  261.000000   question  261.000000  22.0000  22.0000
513   Default  258.000000     lender  258.000000  21.0000  21.0000
507   Default  258.000000       elig  258.000000  20.0000  20.0000
503   Default  258.000000      among  258.000000  19.0000  19.0000
514   Default  258.000000     privat  258.000000  18.0000  18.0000
512   Default  258.000000       lend  258.000000  17.0000  17.0000
515   Default  258.000000     revert  258.000000  16.0000  16.0000
501   Default  258.000000      activ  258.000000  15.0000  15.0000
505   Default  258.000000    context  258.000000  14.0000  14.0000
502   Default  258.000000  agreement  258.000000  13.0000  13.0000
504   Default  258.000000     borrow  258.000000  12.0000  12.0000
516   Default  258.000000      track  258.000000  11.0000  11.0000
517   Default  258.000000   triparti  258.000000  10.0000  10.0000
509   Default  258.000000       fast  258.000000   9.0000   9.0000
510   Default  258.000000        gic  258.000000   8.0000   8.0000
55    Default  539.000000     amount  539.000000   7.0000   7.0000
121   Default  246.000000      swift  246.000000   6.0000   6.0000
5     Default  377.000000       cash  377.000000   5.0000   5.0000
21    Default  330.000000    request  330.000000   4.0000   4.0000
78    Default  251.000000       note  251.000000   3.0000   3.0000
241   Default  182.000000    forward  182.000000   2.0000   2.0000
511   Default  278.000000      hesit  278.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
466   Topic10   19.761757      initi   20.692830   2.7765  -4.6342
462   Topic10   19.760405        ccp   20.692785   2.7764  -4.6343
469   Topic10   19.759111        phi   20.692756   2.7763  -4.6344
464   Topic10   19.757864  demateril   20.692758   2.7763  -4.6344
278   Topic10   16.812951    afrasia   17.736309   2.7690  -4.7958
284   Topic10   16.810713   hongkong   17.736269   2.7689  -4.7960
287   Topic10   16.808132   scblhkhh   17.736230   2.7688  -4.7961
307   Topic10   28.163597     fmsosg   30.517820   2.7422  -4.2799
306   Topic10   28.161030     domest   30.517720   2.7421  -4.2800
482   Topic10   10.898963        fax   11.824235   2.7410  -5.2293
490   Topic10    7.952727  changefax    8.867992   2.7136  -5.5445
286   Topic10   48.266120     nostro   54.237467   2.7059  -3.7412
459   Topic10   25.615235         sr   29.544486   2.6798  -4.3748
279   Topic10   48.266881     arrang   68.149288   2.4775  -3.7412
52    Topic10   58.708086     credit   89.985345   2.3954  -3.5454
43    Topic10   51.131127       code   86.611240   2.2955  -3.6836
75    Topic10   48.263660 

In [20]:
sent_topics_df = pd.DataFrame()

# Get main topic in each document
for i, row in enumerate(lda_model[bow_corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    # Get the Dominant topic, Perc Contribution and Keywords for each document
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:  # => dominant topic
            wp = lda_model.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prop in wp])
            sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
        else:
            break
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

# Add original text to the end of the output
#contents = pd.Series(data)
sent_topics_df = pd.concat([sent_topics_df, data], axis=1)

In [21]:
df_dominant_topic = sent_topics_df
df_dominant_topic.columns = ['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text','TextLength','Tokens','Number of Tokens']

# Show
#df_dominant_topic.head(10)
(df_dominant_topic.iloc[:,0:5]).to_csv("Output.csv",index = False)

In [22]:
# Number of Documents for Each Topic
topic_counts = sent_topics_df['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round((topic_counts/topic_counts.sum())*100, 2)

# Topic Number and Keywords

#topic_num_keywords = sent_topics_df[['Dominant_Topic', 'Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.to_csv("Topic_distribution.csv")

In [23]:
df_dominant_topics

,Num_Documents,Perc_Documents
9.0,323,23.47
8.0,153,11.12
1.0,142,10.32
7.0,123,8.94
5.0,118,8.58
0.0,113,8.21
3.0,108,7.85
2.0,106,7.70
6.0,99,7.19
4.0,91,6.61


In [34]:
data['tokenized']
df_data = data['tokenized']

In [45]:
import missingno as msno
msno.matrix(df_data)

IndexError: tuple index out of range

0       [ganghadharan, ananthakrishnan, andrew, liew, ...
1       [ganghadharan, ananthakrishnan, andrew, liew, ...
2       [secur, settlement, instruct, receiv, isin, cu...
3       [secur, settlement, instruct, receiv, isin, cu...
4       [secur, settlement, instruct, receiv, isin, cu...
5       [secur, take, instruct, convert, share, isin, ...
6       [secur, take, instruct, convert, share, isin, ...
7       [secur, take, instruct, convert, share, isin, ...
8       [safekeep, account, trade, credit, coupon, sec...
9       [safekeep, account, trade, credit, coupon, sec...
10      [safekeep, account, trade, credit, coupon, sec...
11      [safekeep, account, trade, credit, coupon, sec...
12      [bran, ch, repatri, amount, valu, jul, fx, rat...
13      [bran, ch, repatri, amount, valu, jul, fx, rat...
14      [bran, ch, repatri, amount, valu, jul, fx, rat...
15      [rmit, branch, account, scbsb, note, execut, t...
16      [rmit, branch, account, scbsb, note, execut, t...
17      [rmit,